<a href="https://colab.research.google.com/github/TiredCat388/CMSC-173-Midterm-Exam/blob/main/CMSC_173_Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
You are tasked with performing a full data audit and building a robust analytical pipeline. Your final
report should be presented to the lead researchers, who are not data scientists. Your dataset is unique
to you and is located in your exam folder as exam data `202211342.csv.`
#Deliverables
1. Data Quality Audit: A thorough analysis of your data’s issues. Identify and document different
types of errors (e.g., random vs. systematic noise, invalid data formats, logical inconsistencies).

2. Data Cleaning & Preprocessing Strategy: A detailed description of your cleaning method-
ology. Justify your choices. Why did you choose one imputation method over another? How did you handle outliers?

3. Operational Status Prediction: Develop a model to predict the operational status of a panel
(Target Category). Explain your choice of model and evaluate its performance using appropriate
metrics.

4. Energy Output Analysis: Analyze the factors that influence the panel’s energy output (Target Value).
What are the most important predictors? Are there any surprising correlations?
5. Panel Segmentation: Identify and describe any natural groupings of panels in your data. Do
these groups correspond to the different operational statuses or reveal anything about the data
collection issues?
# Key Guiding Questions
- How can you differentiate between random noise and systematic errors that might be present in
your data?
- Is there a discernible pattern in the missing values? What might this imply about the data collection
process?
- The Aux Reading is from a newer, more reliable sensor. How could you leverage this to validate or
correct other readings?
- Some X Coord and Y Coord values appear to be in an unusual format (e.g., hexadecimal strings).
What could this mean, and how should you handle it?
- Do the relationships between the sensor readings and the energy output change for different oper-
ational statuses?
- Are there any distinct groups of panels that might correspond to the different data collection teams
or equipment faults? How would you go about identifying them?

In [ ]:
!git clone https://github.com/TiredCat388/CMSC-173-Midterm-Exam.git
%cd REPOSITORY